In [1]:
import tkp.db  # NOTE: this only works with the queryqueue branch of TraP
import logging
from tkp.db.model import Dataset

logging.basicConfig()

### Database connection settings

In [2]:
engine = 'postgresql'
host = 'localhost'
port = 5433
user = 'gijs'
password = user
database = user
query_loglevel = logging.WARNING  # Set to INFO if you want to see the queries, otherwise use WARNING

### connect to the database

In [3]:
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port, user=user, password=password, database=database)
db.connect()
session = db.Session()

### Execute the query

In [4]:
datasets = session.query(Dataset).all()
[d.description for d in datasets]


[u'TRAP dataset']